## Transformation of PGN format for MBA: 

#### the association rules requires a sparse matrix in order to run to completion.
#### 

#### Install required python libraries

In [2]:
%pip install mlxtend pandas sklearn


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a Python library that isn't in Homebrew,
    use a virtual environment:
    
    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz
    
    If you wish to install a Python application that isn't in Homebrew,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. You can install pipx with
    
    brew install pipx
    
    You may restore the old behavior of pip by passing
    the '--break-system-packages' flag to pip, or by adding
    'break-system-packages = true' to your pip.conf file. The latter
    will permanently disable this erro

#### Import libraries for required functionality

In [3]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules, fpmax, fpgrowth
from mlxtend.preprocessing import TransactionEncoder

#### We need to generalize groups of moves into baskets.  Coincedently, each game of chess breaks up like a story into a beginning or openning, middle and end.  These three parts is just one way of breaking up the game, within each of these groups further subdivisions will help to create more sparse transactions.  The results of each game maps via association to each layer or grouping.  

#### The edit distance metric via a dynamic programming algorithm assists with generalizing specific moves to a limited set of 384 different possible moves or products for the basket / transactions.

In [4]:
def edit_distance(predefined_set, actual_move):
    m, n = len(predefined_set), len(actual_move)
    dynamic_programming_table = [[0] * (n + 1) for _ in range(m + 1)]

    # Initialize the dynamic_programming_table
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dynamic_programming_table[i][j] = j  # Insert characters of actual_move
            elif j == 0:
                dynamic_programming_table[i][j] = i  # Remove characters of predefined_set
            elif predefined_set[i - 1] == actual_move[j - 1]:
                dynamic_programming_table[i][j] = dynamic_programming_table[i - 1][j - 1]  # match
            else:
                dynamic_programming_table[i][j] = 1 + min(
                                   dynamic_programming_table[i - 1][j - 1], # Substitue
                                   dynamic_programming_table[i][j - 1],    # Insert
                                   dynamic_programming_table[i - 1][j]    # Remove
                                   )
    return dynamic_programming_table[m][n]

#### Verify or set our expectations of the edit distance metrics:

In [5]:
possible_chess_moves = "Ba1 Ba2 Ba3 Ba4 Ba5 Ba6 Ba7 Ba8 Bb1 Bb2 Bb3 Bb4 Bb5 Bb6 Bb7 Bb8 Bc1 Bc2 Bc3 Bc4 Bc5 Bc6 Bc7 Bc8 Bd1 Bd2 Bd3 Bd4 Bd5 Bd6 Bd7 Bd8 Be1 Be2 Be3 Be4 Be5 Be6 Be7 Be8 Bf1 Bf2 Bf3 Bf4 Bf5 Bf6 Bf7 Bf8 Bg1 Bg2 Bg3 Bg4 Bg5 Bg6 Bg7 Bg8 Bh1 Bh2 Bh3 Bh4 Bh5 Bh6 Bh7 Bh8 Ka1 Ka2 Ka3 Ka4 Ka5 Ka6 Ka7 Ka8 Kb1 Kb2 Kb3 Kb4 Kb5 Kb6 Kb7 Kb8 Kc1 Kc2 Kc3 Kc4 Kc5 Kc6 Kc7 Kc8 Kd1 Kd2 Kd3 Kd4 Kd5 Kd6 Kd7 Kd8 Ke1 Ke2 Ke3 Ke4 Ke5 Ke6 Ke7 Ke8 Kf1 Kf2 Kf3 Kf4 Kf5 Kf6 Kf7 Kf8 Kg1 Kg2 Kg3 Kg4 Kg5 Kg6 Kg7 Kg8 Kh1 Kh2 Kh3 Kh4 Kh5 Kh6 Kh7 Kh8 Na1 Na2 Na3 Na4 Na5 Na6 Na7 Na8 Nb1 Nb2 Nb3 Nb4 Nb5 Nb6 Nb7 Nb8 Nc1 Nc2 Nc3 Nc4 Nc5 Nc6 Nc7 Nc8 Nd1 Nd2 Nd3 Nd4 Nd5 Nd6 Nd7 Nd8 Ne1 Ne2 Ne3 Ne4 Ne5 Ne6 Ne7 Ne8 Nf1 Nf2 Nf3 Nf4 Nf5 Nf6 Nf7 Nf8 Ng1 Ng2 Ng3 Ng4 Ng5 Ng6 Ng7 Ng8 Nh1 Nh2 Nh3 Nh4 Nh5 Nh6 Nh7 Nh8 Qa1 Qa2 Qa3 Qa4 Qa5 Qa6 Qa7 Qa8 Qb1 Qb2 Qb3 Qb4 Qb5 Qb6 Qb7 Qb8 Qc1 Qc2 Qc3 Qc4 Qc5 Qc6 Qc7 Qc8 Qd1 Qd2 Qd3 Qd4 Qd5 Qd6 Qd7 Qd8 Qe1 Qe2 Qe3 Qe4 Qe5 Qe6 Qe7 Qe8 Qf1 Qf2 Qf3 Qf4 Qf5 Qf6 Qf7 Qf8 Qg1 Qg2 Qg3 Qg4 Qg5 Qg6 Qg7 Qg8 Qh1 Qh2 Qh3 Qh4 Qh5 Qh6 Qh7 Qh8 Ra1 Ra2 Ra3 Ra4 Ra5 Ra6 Ra7 Ra8 Rb1 Rb2 Rb3 Rb4 Rb5 Rb6 Rb7 Rb8 Rc1 Rc2 Rc3 Rc4 Rc5 Rc6 Rc7 Rc8 Rd1 Rd2 Rd3 Rd4 Rd5 Rd6 Rd7 Rd8 Re1 Re2 Re3 Re4 Re5 Re6 Re7 Re8 Rf1 Rf2 Rf3 Rf4 Rf5 Rf6 Rf7 Rf8 Rg1 Rg2 Rg3 Rg4 Rg5 Rg6 Rg7 Rg8 Rh1 Rh2 Rh3 Rh4 Rh5 Rh6 Rh7 Rh8 a1 a2 a3 a4 a5 a6 a7 a8 b1 b2 b3 b4 b5 b6 b7 b8 c1 c2 c3 c4 c5 c6 c7 c8 d1 d2 d3 d4 d5 d6 d7 d8 e1 e2 e3 e4 e5 e6 e7 e8 f1 f2 f3 f4 f5 f6 f7 f8 g1 g2 g3 g4 g5 g6 g7 g8 h1 h2 h3 h4 h5 h6 h7 h8"
tests="""1.d4 d5 2.c4 c6 3.Nc3 Nf6 4.e3 a6 5.Bd3 b5 6.cxd5 cxd5 7.Bd2 e6 8.Nf3 Bd6
9.Ne5 Nbd7 10.f4 Bb7 11.O-O O-O 12.Rc1 Nb6 13.Qf3 Rc8 14.Nb1 Qe7 15.b3 Nc4
16.Bc3 Na3 17.Bb2 Nxb1 18.Bxb1 Ne4 19.Qe2 f6 20.Nd3 Bb4 1/2-1/2"""
print (tests)
tcs =tests.split()
pcm = possible_chess_moves.split()
for t in tcs:
  min_distance = 100
  for p in pcm:
    if edit_distance(p,t) <= min_distance:
      min_distance = edit_distance(p,t)
      print (f"Possible move: {p} - Actual move: {t} - Distance: {edit_distance(p,t)}") 

1.d4 d5 2.c4 c6 3.Nc3 Nf6 4.e3 a6 5.Bd3 b5 6.cxd5 cxd5 7.Bd2 e6 8.Nf3 Bd6
9.Ne5 Nbd7 10.f4 Bb7 11.O-O O-O 12.Rc1 Nb6 13.Qf3 Rc8 14.Nb1 Qe7 15.b3 Nc4
16.Bc3 Na3 17.Bb2 Nxb1 18.Bxb1 Ne4 19.Qe2 f6 20.Nd3 Bb4 1/2-1/2
Possible move: Ba1 - Actual move: 1.d4 - Distance: 4
Possible move: Ba2 - Actual move: 1.d4 - Distance: 4
Possible move: Ba3 - Actual move: 1.d4 - Distance: 4
Possible move: Ba4 - Actual move: 1.d4 - Distance: 3
Possible move: Bb4 - Actual move: 1.d4 - Distance: 3
Possible move: Bc4 - Actual move: 1.d4 - Distance: 3
Possible move: Bd1 - Actual move: 1.d4 - Distance: 3
Possible move: Bd2 - Actual move: 1.d4 - Distance: 3
Possible move: Bd3 - Actual move: 1.d4 - Distance: 3
Possible move: Bd4 - Actual move: 1.d4 - Distance: 2
Possible move: Kd4 - Actual move: 1.d4 - Distance: 2
Possible move: Nd4 - Actual move: 1.d4 - Distance: 2
Possible move: Qd4 - Actual move: 1.d4 - Distance: 2
Possible move: Rd4 - Actual move: 1.d4 - Distance: 2
Possible move: d4 - Actual move: 1.d4 - Dista

##### This test reveals that we need to further split on the dot of the move number and add the three possible results to our list of possible chess moves.

#### We now need to build the representation of our Basket:

In [6]:
class Basket:
    possible_chess_moves = "Ba1 Ba2 Ba3 Ba4 Ba5 Ba6 Ba7 Ba8 Bb1 Bb2 Bb3 Bb4 Bb5 Bb6 Bb7 Bb8 Bc1 Bc2 Bc3 Bc4 Bc5 Bc6 Bc7 Bc8 Bd1 Bd2 Bd3 Bd4 Bd5 Bd6 Bd7 Bd8 Be1 Be2 Be3 Be4 Be5 Be6 Be7 Be8 Bf1 Bf2 Bf3 Bf4 Bf5 Bf6 Bf7 Bf8 Bg1 Bg2 Bg3 Bg4 Bg5 Bg6 Bg7 Bg8 Bh1 Bh2 Bh3 Bh4 Bh5 Bh6 Bh7 Bh8 Ka1 Ka2 Ka3 Ka4 Ka5 Ka6 Ka7 Ka8 Kb1 Kb2 Kb3 Kb4 Kb5 Kb6 Kb7 Kb8 Kc1 Kc2 Kc3 Kc4 Kc5 Kc6 Kc7 Kc8 Kd1 Kd2 Kd3 Kd4 Kd5 Kd6 Kd7 Kd8 Ke1 Ke2 Ke3 Ke4 Ke5 Ke6 Ke7 Ke8 Kf1 Kf2 Kf3 Kf4 Kf5 Kf6 Kf7 Kf8 Kg1 Kg2 Kg3 Kg4 Kg5 Kg6 Kg7 Kg8 Kh1 Kh2 Kh3 Kh4 Kh5 Kh6 Kh7 Kh8 Na1 Na2 Na3 Na4 Na5 Na6 Na7 Na8 Nb1 Nb2 Nb3 Nb4 Nb5 Nb6 Nb7 Nb8 Nc1 Nc2 Nc3 Nc4 Nc5 Nc6 Nc7 Nc8 Nd1 Nd2 Nd3 Nd4 Nd5 Nd6 Nd7 Nd8 Ne1 Ne2 Ne3 Ne4 Ne5 Ne6 Ne7 Ne8 Nf1 Nf2 Nf3 Nf4 Nf5 Nf6 Nf7 Nf8 Ng1 Ng2 Ng3 Ng4 Ng5 Ng6 Ng7 Ng8 Nh1 Nh2 Nh3 Nh4 Nh5 Nh6 Nh7 Nh8 Qa1 Qa2 Qa3 Qa4 Qa5 Qa6 Qa7 Qa8 Qb1 Qb2 Qb3 Qb4 Qb5 Qb6 Qb7 Qb8 Qc1 Qc2 Qc3 Qc4 Qc5 Qc6 Qc7 Qc8 Qd1 Qd2 Qd3 Qd4 Qd5 Qd6 Qd7 Qd8 Qe1 Qe2 Qe3 Qe4 Qe5 Qe6 Qe7 Qe8 Qf1 Qf2 Qf3 Qf4 Qf5 Qf6 Qf7 Qf8 Qg1 Qg2 Qg3 Qg4 Qg5 Qg6 Qg7 Qg8 Qh1 Qh2 Qh3 Qh4 Qh5 Qh6 Qh7 Qh8 Ra1 Ra2 Ra3 Ra4 Ra5 Ra6 Ra7 Ra8 Rb1 Rb2 Rb3 Rb4 Rb5 Rb6 Rb7 Rb8 Rc1 Rc2 Rc3 Rc4 Rc5 Rc6 Rc7 Rc8 Rd1 Rd2 Rd3 Rd4 Rd5 Rd6 Rd7 Rd8 Re1 Re2 Re3 Re4 Re5 Re6 Re7 Re8 Rf1 Rf2 Rf3 Rf4 Rf5 Rf6 Rf7 Rf8 Rg1 Rg2 Rg3 Rg4 Rg5 Rg6 Rg7 Rg8 Rh1 Rh2 Rh3 Rh4 Rh5 Rh6 Rh7 Rh8 a1 a2 a3 a4 a5 a6 a7 a8 b1 b2 b3 b4 b5 b6 b7 b8 c1 c2 c3 c4 c5 c6 c7 c8 d1 d2 d3 d4 d5 d6 d7 d8 e1 e2 e3 e4 e5 e6 e7 e8 f1 f2 f3 f4 f5 f6 f7 f8 g1 g2 g3 g4 g5 g6 g7 g8 h1 h2 h3 h4 h5 h6 h7 h8 1-0 1/2 0-1"
    hm={}
    pcm = []
    def __init__(self):
        self.pcm = self.possible_chess_moves.split()
        print(f"Length of Products:{len(self.pcm)}")
        for pm in self.pcm:
            self.hm.update({pm:[False]})

    def metric_lookup(self, query):
        min_metric = 10
        for m in self.pcm:
            metric = edit_distance(query,m)
            if metric <= min_metric: # < for the first tie; <= for the last tie
                min_metric = metric
                s2m = m
        return s2m, min_metric

    def update_basket(self, item, move_number=0):
        tempList=self.hm.get(item)
        # detect the result of the move
        temp_list_res01 = self.hm.get(self.pcm[len(self.pcm)-1])
        temp_list_res12 = self.hm.get(self.pcm[len(self.pcm)-2])
        temp_list_res10 = self.hm.get(self.pcm[len(self.pcm)-3])
        if item == self.pcm[len(self.pcm)-1] or item == self.pcm[len(self.pcm)-2] or item == self.pcm[len(self.pcm)-3]:
            tmnr = move_number
            # apply the results across all prior transactions
            while not temp_list_res01[tmnr] and not temp_list_res12[tmnr] and not temp_list_res10[tmnr]:
                tempList[tmnr] = True
                if tmnr > 0:
                    tmnr -= 1
        # make room for a new transaction across all products
        while move_number >= len(tempList):
            for pm in self.pcm:
                tl=self.hm.get(pm)
                tl.append(False)
                self.hm.update({pm:tl})
            tempList=self.hm.get(item)
        if not tempList[ move_number ] :
            tempList[ move_number ]=True #1+tempList[ move_number ]
            self.hm.update({item:tempList})

    def __str__(self):
        ret = ""
        for k in self.hm.keys():
            d=""
            for elm in self.hm.get(k):
                if elm:
                    d+=" "+str(elm)+" "
                else:
                    d+=" "+str(elm)
            ret+=str(f"{k}:{d}\n")
        return ret

    def annalyze_apriori(self):
        df = pd.DataFrame(self.hm)
        # Generate frequent itemsets
        frequent_itemsets = apriori(df, min_support=0.001, use_colnames=True)
        # Generate association rules
        #rules = association_rules(frequent_itemsets, support_only=True, min_threshold=0.001)
        rules = association_rules(frequent_itemsets, metric='support', min_threshold=0.001)
        print(rules)

#### We now build out various levels of abstractions via transaction deliminators representative of and according to our notions of chess strategy:

In [7]:
test1="1.d4 d5 2.c4 c6 3.Nc3 Nf6 4.e3 a6 5.Bd3 b5 6.cxd5 cxd5 7.Bd2 e6 8.Nf3 Bd6 9.Ne5 Nbd7 10.f4 Bb7 11.O-O O-O 12.Rc1 Nb6 13.Qf3 Rc8 14.Nb1 Qe7 15.b3 Nc4 16.Bc3 Na3 17.Bb2 Nxb1 18.Bxb1 Ne4 19.Qe2 f6 20.Nd3 Bb4 1/2-1/2"
test2="1.d4 d5 2.c4 c6 3.Nc3 Nf6 4.e3 a6 5.Bd3 b5 6.cxd5 cxd5 7.Bd2 e6 8.Nf3 Bd6 9.Ne5 Nbd7 10.f4 Bb7 11.O-O O-O 12.Rc1 Nb6 13.Qf3 Rc8 14.Nb1 Qe7 15.b3 Nc4 16.Bc3 Na3 17.Bb2 Nxb1 18.Bxb1 Ne4 19.Qe2 f6 20.Nd3 Bb4 1-0"
tests=test1+" "+test2
print (tests)
tcs =tests.split()
mba = Basket()
# control over the relative degree of sparsity in each basket
move_number=0 # appends to list on each pair of moves
pawn_move_number=0 # appends to list on each pawn move: a bit more dense than the move_number
king_move_number=0 # appends to list on each king move more sparse at the end of a game
queen_move_number=0 # appends to list on each queen move more sparse in the middle of a game
castle_move_number=0 # appends to list splits the opening from the rest of the game 
for t in tcs:
    T=t.split('.')
    s=T[0]
    if len(T) > 1:
       move_number+=1  # appends to list
       if s == "1":
         king_move_number+=1 # appends to list
         queen_move_number+=1 # appends to list
         castle_move_number+=1
         if len(T[1]) > 2:
           pawn_move_number+=1
       s=T[1]
    if s.startswith('Q'):
       queen_move_number +=1
    elif s.startswith('K'):
       king_move_number +=1
    elif s.startswith('O'):
       castle_move_number +=1
       king_move_number +=1
    if s== "O-O":
       s="Rf8"
       if len(t) >3:
          s="Rf1"
       s2m, min_metric = mba.metric_lookup(s)
       mba.update_basket(s2m, pawn_move_number)
       s="Kg8"
       if len(t) >3:
          s="Kg1"
    if s== "O-O-O":
       s="Rd8"
       if len(t) >5:
          s="Rd1"
       s2m, min_metric = mba.metric_lookup(s)
       mba.update_basket(s2m, pawn_move_number)
       s="Kc8"
       if len(t) >5:
          s="Kc1"
    s2m, min_metric = mba.metric_lookup(s)
    if 2 == len(s2m):
        pawn_move_number+=1
    mba.update_basket(s2m, pawn_move_number)
    
print(mba)


1.d4 d5 2.c4 c6 3.Nc3 Nf6 4.e3 a6 5.Bd3 b5 6.cxd5 cxd5 7.Bd2 e6 8.Nf3 Bd6 9.Ne5 Nbd7 10.f4 Bb7 11.O-O O-O 12.Rc1 Nb6 13.Qf3 Rc8 14.Nb1 Qe7 15.b3 Nc4 16.Bc3 Na3 17.Bb2 Nxb1 18.Bxb1 Ne4 19.Qe2 f6 20.Nd3 Bb4 1/2-1/2 1.d4 d5 2.c4 c6 3.Nc3 Nf6 4.e3 a6 5.Bd3 b5 6.cxd5 cxd5 7.Bd2 e6 8.Nf3 Bd6 9.Ne5 Nbd7 10.f4 Bb7 11.O-O O-O 12.Rc1 Nb6 13.Qf3 Rc8 14.Nb1 Qe7 15.b3 Nc4 16.Bc3 Na3 17.Bb2 Nxb1 18.Bxb1 Ne4 19.Qe2 f6 20.Nd3 Bb4 1-0
Length of Products:387
Ba1: False False False False False False False False False False False False False False False False False False False False False False False False False False False
Ba2: False False False False False False False False False False False False False False False False False False False False False False False False False False False
Ba3: False False False False False False False False False False False False False False False False False False False False False False False False False False False
Ba4: False False False False False False False False F

In [8]:
mba.annalyze_apriori()
# The output of the apriori algorithm will be printed here

        antecedents                                        consequents  \
0             (Bb2)                                              (Bb1)   
1             (Bb1)                                              (Bb2)   
2             (Bb1)                                              (Bc3)   
3             (Bc3)                                              (Bb1)   
4             (Na3)                                              (Bb1)   
...             ...                                                ...   
2729225       (Rf8)  (Kg8, f4, Rf1, Kg1, Rc8, Qe7, Qf3, Nb6, Rc1, N...   
2729226       (Rc1)  (Kg8, f4, Rf1, Kg1, Rc8, Qe7, Qf3, Nb6, Rf8, N...   
2729227       (Nb1)  (Kg8, f4, Rf1, Kg1, Rc8, Qe7, Qf3, Nb6, Rf8, R...   
2729228       (1/2)  (Kg8, f4, Rf1, Kg1, Rc8, Qe7, Qf3, Nb6, Rf8, R...   
2729229       (Bb7)  (Kg8, f4, Rf1, Kg1, Rc8, Qe7, Qf3, Nb6, Rf8, R...   

         antecedent support  consequent support   support  confidence  \
0                  0.074074           

#### These rules show the time/space-complexity for a very small set of games.  We can see that the more densely packed our transactions are the more time and space will be required to compute the associations.  We can now move on to applying the pgn transformation over our complete collection of games with greater control over the relative sparseness for the computation of association rules.  We have included transaction delimeters that map nicely to our understanding of movement / progress in chess game development from the opening, through the middle and into the end game.  These divisions help to reduce the complexity in the rule building process.